In [1]:
import pandas as pd
df = pd.read_pickle('kbo_raw.pkl')

In [2]:
len(df)

210974

In [3]:
df = df[df['datetime']>'20150801235959']
df['datetime'] = pd.to_datetime(df['datetime'])

In [5]:
df.to_pickle('kbo_raw.pkl')

In [6]:
df.drop(['url','org_url','desc'], axis = 1, inplace = True)

In [7]:
df['title']=df['title'].str.replace(r"[^0-9a-zA-Z가-힣,.!?@=<>()\[\]{} ]+", " ")

In [13]:
docs=df['title'].str.strip() + ". " + df['contents']

In [14]:
docs[:10]

0    아쉬운 마리한화... 그러나 박수가 아깝지 않다. [KBO리그] 전체적 전력 미흡으...
1    케이티, 신생팀 최다승 타이 한화 PS 좌절. (서울=연합뉴스) 천병혁 기자 = 신...
2    [프로야구]화제의 중심 한화, 결국 포스트시즌 탈락. 한화 이글스 선수단. 2015...
3    성공과 실패의 경계선 위에 선 2015년 한화. 한화 이글스 선수단. (연합뉴스 자...
4    [PS 진출실패] 한화에게 뼈아팠던  아쉬운 세 경기. [엑스포츠뉴스=이지은 기자]...
5    [PS 진출실패] 한화, 그래도 2016년이 기대되는 이유. [엑스포츠뉴스=이지은 ...
6    한화이글스, 아쉬운 탈락  8년 연속 PS 실패. 사진=방송화면/한화이글스한화이글스...
7    PS 탈락  한화 김태균  약속 못 지켜 죄송, 마음 무겁다. [마이데일리 = 수원...
8    총력전 선언  한화, 로저스 나올 상황조차 없었다. [매경닷컴 MK스포츠(수원) 강...
9    [PS 진출 실패] 키워드로 본 한화의 가을 도전. [엑스포츠뉴스=박진태 기자] 한...
dtype: object

In [15]:
# 문장 내용과 관계없는 특수문자 제거
test=docs.str.replace(r"[^0-9a-zA-Z가-힣=,.:!?@<>()/\-\[\]{} ]+", " ")
# 이메일 제거 및 url 제거 #\([a-z.:/ ]+\)|
test=test.str.replace(r"([0-9a-zA-Z.]+@[0-9a-zA-Z.]+)|[0-9a-zA-Z.]+.kr|[0-9a-zA-Z.]+.com","")
test=test.str.replace(r"http:[/]+","")
#remove byline
test=test.str.replace(r"\[[0-9a-zA-Z가-힣/=()!|,. ]+\]|\([0-9a-zA-Z가-힣()!|,. ]+=[0-9a-zA-Z가-힣()!|,. ]+\)", " ")
#비어있는 () 및 언론사 종특 제거
test=test.str.replace(r"\(\)[. ]|마이데일리.|조이뉴스24.", " ")
#알파벳으로 끝나는 문장 제거
#test=test.str.replace(r"[a-zA-Z ]+[,.]", "")
#test=test.str.replace(r"([0-9a-zA-Z.]+@[0-9a-zA-Z.]+)|[0-9a-zA-Z.]+[.kr]|[0-9a-zA-Z.]+[.com]","")
#sentences=test.str.replace(r"\[[0-9a-zA-Z가-힣()|,. ]+\]|([0-9a-zA-Z.]+@[0-9a-zA-Z.]+)
#|[0-9a-zA-Z.]+[.kr]|[0-9a-zA-Z.]+[.com]|[0-9a-zA-Z.]+@|\([a-z.:/ ]+\)|마이데일리.|조이뉴스24.|[a-zA-Z,. ]+\.|기사제공 [0-9a-zA-Z가-힣 .]+", " ")

## 문장추출

In [16]:
doc_sentences=test.str.findall(r"[0-9a-zA-Z가-힣%,()\- ]+\. ")

In [17]:
len(doc_sentences)

198064

In [29]:
%%time
from konlpy.tag import Mecab; m = Mecab()
pos = lambda d: ['/'.join(p) for p in m.pos(d)]

tagged_sentences=doc_sentences.apply(lambda x:pos(''.join(x)))

CPU times: user 7min 1s, sys: 3.7 s, total: 7min 5s
Wall time: 7min 5s


In [32]:
words_count=tagged_sentences.apply(lambda x:len(x))

In [39]:
df['sentences']=doc_sentences
df['tagged_sentences']=tagged_sentences
df['words_count']=words_count

In [40]:
df2017=df[df['datetime']>'20161231235959']

In [41]:
len(df2017)

69622

In [33]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

In [42]:
p2 = figure(plot_width=600, plot_height=600,title='KBO Word2Vec', x_axis_type='datetime')

p2.circle(df2017['datetime'].tolist(), df2017['words_count'].tolist(), size=3, color="navy", alpha=0.5)
show(p2)

In [44]:
df.drop(['contents','title'], axis = 1, inplace = True)

In [45]:
df

,level_0,datetime,sentences,tagged_sentences,words_count
0,HH,2015-10-03 20:43:03,"[ 그러나 박수가 아깝지 않다. , 한화 이글스가 2009년부터 2014년까지 6...","[그러나/MAJ, 박수/NNG, 가/JKS, 아깝/VA, 지/EC, 않/VX, 다/...",802
1,HH,2015-10-03 16:47:21,"[케이티, 신생팀 최다승 타이 한화 PS 좌절. , 신생팀 케이티 위즈가 통렬한 ...","[케이티/NNP, ,/SC, 신생/NNG, 팀/NNG, 최/XPN, 다승/NNG, ...",384
2,HH,2015-10-03 16:50:42,"[ 화제의 중심 한화, 결국 포스트시즌 탈락. , 한화 이글스 선수단. , 이로써 ...","[화제/NNG, 의/JKG, 중심/NNG, 한화/NNG, ,/SC, 결국/MAG, ...",715
3,HH,2015-10-03 16:57:10,"[성공과 실패의 경계선 위에 선 2015년 한화. , 한화 이글스 선수단. , 2...","[성공/NNG, 과/JC, 실패/NNG, 의/JKG, 경계선/NNG, 위/NNG, ...",627
4,HH,2015-10-03 17:00:04,"[ 한화에게 뼈아팠던 아쉬운 세 경기. , 후회는 이미 늦다. , 하지만 분...","[한화/NNG, 에게/JKB, 뼈/NNG, 아팠/VA+EP, 던/ETM, 아쉬운/V...",1070
5,HH,2015-10-03 17:00:04,"[ 한화, 그래도 2016년이 기대되는 이유. , 아등바등 싸워온 한 시즌이 ...","[한화/NNG, ,/SC, 그래도/MAJ, 2016/SN, 년/NNBC, 이/JKS...",1117
6,HH,2015-10-03 17:03:04,"[한화이글스, 아쉬운 탈락 8년 연속 PS 실패. , 한화이글스한화이글스가 결국 ...","[한화이글스/NNP, ,/SC, 아쉬운/VA+ETM, 탈락/NNG, 8/SN, 년/...",210
7,HH,2015-10-03 17:08:04,"[PS 탈락 한화 김태균 약속 못 지켜 죄송, 마음 무겁다. , 팬들께 죄...","[PS/SL, 탈락/MAG, 한화/NNG, 김태균/NNP, 약속/NNG, 못/MAG...",324
8,HH,2015-10-03 17:33:09,"[총력전 선언 한화, 로저스 나올 상황조차 없었다. , 에이스까지 모든 투수가...","[총력전/NNG, 선언/NNG, 한화/NNG, ,/SC, 로저스/NNP, 나올/VV...",383
9,HH,2015-10-03 16:43:03,"[ 키워드로 본 한화의 가을 도전. , 한화 이글스의 가을 야구 도전이 막을 ...","[키워드/NNP, 로/JKB, 본/VV+ETM, 한화/NNG, 의/JKG, 가을/N...",967


In [47]:
df.reset_index(level=0, inplace=True)

In [49]:
df.drop(['index'], axis = 1, inplace = True)

In [51]:
df.to_pickle('kbo_parsed.pkl')

In [90]:
df['title']=df['title'].str.replace(r"[^0-9a-zA-Z가-힣,.!?@=<>()\[\]{} ]+", " ")
df['contents']=df['contents'].str.replace(r"[^0-9a-zA-Z가-힣,.!?@=<>()\[\]{} ]+|[0-9a-zA-Z.]+.kr|[0-9a-zA-Z.]+.com|[0-9a-zA-Z.]+@[a-zA-Z.]+|[0-9a-zA-Z.]+@|\([a-z.:/]+\)|마이데일리.|조이뉴스24.|[a-zA-Z,. ]+\.|기사제공 [0-9a-zA-Z가-힣 .]", " ")

In [96]:
df['contents']=df['contents'].str.findall(r"[0-9a-zA-Z가-힣,.!?<>() ]+\. ").apply(lambda x: ' '.join(x))